In [1]:
import numpy as np
import random
import math
import csv
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorboard
from keras.optimizers import SGD

/home/adrien/anaconda3/envs/GPI770/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/adrien/anaconda3/envs/GPI770/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/adrien/anaconda3/envs/GPI770/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/adrien/anaconda3/envs/GPI770/lib/python3

In [2]:
def read_csv(url):
    """
    Args:
        url (string): the url of the file
    Returns:
        df: the dataframe filled
    """
    df = pd.read_csv(url, header=None)
    df.head()
    return df

# Réseaux neuronaux avec TensorFlow 
Multi-Layer Perceptron

| Hyperparametre | valeur de base   |
|------|------|
|   Nombre de couches total | 4, y compris la couche d’entrée (input) et la couche de sortie (output) |
|   Nombre de perceptrons dans la couche cachée (hidden layer) | 100, 100, 2 |
|   Nombre d’itérations (epochs) | 60 |
|   Taux d’apprentissage (learningrate) | 0.0005 |
|   Batch size | 100 |

In [3]:
nb_layers = 4
nb_perceptron = [100,100,2]
nb_iteration = 60
learning_rate = 0.0005
batch_size = 100

## Version longue
1. Faire for loop pour Avec la librairie TensorFlow (avec l’API tf.keras) 
    1. Vous pouvez vous inspirer de la littérature, de recherches Internet, de la documentation de TensorFlow ou d’autres sources
    2. Ce modèle doit être original
    3. Il s’agit ici de construire un Multi-Layer Perceptron model.
    4. Faites attention aux nombres de perceptrons à l’entrée et à la sortie; la première valeur doit concorder avec le  nombre de caractéristiques [features] que votre vecteur en entrée comporte [son nombre de dimensions] alors que la deuxième doit concorder avec le nombre de classes que vous avez à la sortie.
        
2. Dans votre code source, vous devrez ajouter le code nécessaire à l’outil TensorBoard (TensorFlow) Cet outil vous permettra alors de suivre l’évolution des hyperparamètres, via un serveur Web local sur votre machine, et vous facilitera grandement  la tâche dans votre étude des réseaux neuronaux. Vous devez ajouter le code permettant de capturer les valeurs de la précision [accuracy] et la mesure de l’inconsistance entre la valeur prédite et la vraie valeur [loss].
3. Voir tableau Hyperparametre, vous devez sélectionner trois valeurs différentes et lancer l’apprentissage avec ces valeurs. **Vous aurez alors 12 modèles d’apprentissages différents**. Notez les résultats de la précision [accuracy] que vous avez obtenue au final. À la fin de vos manipulations, vous devriez avoir, par le biais de TensorBoard, des graphiques comme ci-dessous décrivant la précision [accuracy] et le loss. 
    1. *Aller voir enoncé pour exemples de graph*

## Version courte
1. Faire une for loop qui joue avec les hyperparametres
    1. Creer un reseau de neuronne avec les hyperparametres
    2. Utiliser TensorBoard pour capturer les valeurs de précision, de loss, le score F1 de chaque reseau
    3. Prenez en note le temps d’exécution de vos modèles autant en phase d’apprentissage qu’en phase de test.
    4. Afficher les graphiques

In [4]:
def OneHot(x):
    if x == 1:
        res = [1,0]
    elif x == 0:
        res = [0,0]
    return res

In [6]:
# read the csv and get the dataframe
df = read_csv("../galaxy_feature_vectors.csv")

# replace the output by a binairy vector 
new_labels = []
for i in df[75]:
    i = OneHot(i)
    new_labels.append(i)

# get the train and test sample
df = df.drop(0, axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(df.loc[:, df.columns != 75], new_labels, test_size=0.2,stratify=df[75])

In [7]:
# Set labels as DataFrame
Y_train = pd.DataFrame(Y_train)
Y_test = pd.DataFrame(Y_test)

In [9]:
#Split the Train set into train and validate set
nx_train, nx_val, ny_train, ny_val = train_test_split(X_train, Y_train, test_size=0.2,stratify=Y_train)

In [10]:
#Create the model 
model = Sequential()

In [11]:
#First hidden layer with 100 neurons 
model.add(Dense(units=100, activation='sigmoid', input_dim = 74))

#Second hidden layer with 100 neurons 
model.add(Dense(units=100, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#Last layer, the activation layer with 2 outputs
model.add(Dense(units = 2, activation='sigmoid'))

In [13]:
#Compile the model
sgd = SGD(lr=learning_rate)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [14]:
#3. Entraîner 
log_dir  = './logs/nn_64'
# On va utiliser Tensorboard pour visualizer le progrès de l'entraînement
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(nx_train, ny_train, validation_data=(nx_val, ny_val),
          epochs=60, batch_size=100, callbacks=[tb_callback])

Instructions for updating:
Use tf.cast instead.
Train on 10820 samples, validate on 2706 samples
Epoch 1/60
10820/10820 [==============================] - 0s 28us/step - loss: 0.5165 - accuracy: 0.7405 - val_loss: 0.5049 - val_accuracy: 0.7404
Epoch 2/60
10820/10820 [==============================] - 0s 24us/step - loss: 0.4904 - accuracy: 0.7405 - val_loss: 0.4770 - val_accuracy: 0.7404
Epoch 3/60
10820/10820 [==============================] - 0s 21us/step - loss: 0.4664 - accuracy: 0.7405 - val_loss: 0.4565 - val_accuracy: 0.7406
Epoch 4/60
10820/10820 [==============================] - 0s 21us/step - loss: 0.4486 - accuracy: 0.7405 - val_loss: 0.4411 - val_accuracy: 0.7406
Epoch 5/60
10820/10820 [==============================] - 0s 20us/step - loss: 0.4350 - accuracy: 0.7405 - val_loss: 0.4292 - val_accuracy: 0.7406
Epoch 6/60
10820/10820 [==============================] - 0s 20us/step - loss: 0.4244 - accuracy: 0.7405 - val_loss: 0.4198 - val_accuracy: 0.7406
Epoch 7/60
10820/1082

10820/10820 [==============================] - 0s 35us/step - loss: 0.3566 - accuracy: 0.7595 - val_loss: 0.3564 - val_accuracy: 0.7596
Epoch 55/60
10820/10820 [==============================] - 0s 26us/step - loss: 0.3564 - accuracy: 0.7595 - val_loss: 0.3563 - val_accuracy: 0.7596
Epoch 56/60
10820/10820 [==============================] - 0s 32us/step - loss: 0.3562 - accuracy: 0.7595 - val_loss: 0.3561 - val_accuracy: 0.7596
Epoch 57/60
10820/10820 [==============================] - 0s 19us/step - loss: 0.3560 - accuracy: 0.7595 - val_loss: 0.3559 - val_accuracy: 0.7596
Epoch 58/60
10820/10820 [==============================] - 0s 20us/step - loss: 0.3558 - accuracy: 0.7595 - val_loss: 0.3557 - val_accuracy: 0.7596
Epoch 59/60
10820/10820 [==============================] - 0s 20us/step - loss: 0.3557 - accuracy: 0.7595 - val_loss: 0.3556 - val_accuracy: 0.7596
Epoch 60/60
10820/10820 [==============================] - 0s 19us/step - loss: 0.3555 - accuracy: 0.7595 - val_loss: 0.3554

In [15]:
#4 Evaluer le modèle
score = model.evaluate(X_test, Y_test)
print(score)

3382/3382 [==============================] - 0s 22us/step
[0.3554232397740723, 0.7594618797302246]


# Machines à vecteur de support 

## Version Longue
1. À l’aide de la librairie scikit-learn, produisez un code source permettant de classifier les galaxies avec l’aide de l’algorithme SVM.
2. Plusieurs modèles calculés à l’aide de SVM avec différentes valeurs d’hyperparamètres sont demandés. Fiez-vous aux matrices ci-dessous afin de les réaliser

Tableau 5.2 : Matrice des hyperparemètres - SVM kernel = ‘linear’

| Poids des classes / Variable C  | class_weight= {‘balanced’}   | 
|------|------|
|   1E-03 | X | 
|   1E-01 | X | 
|   1.0 | X | 
|   10.0 | X | 

Tableau 5.3 : Matrice des hyperparemètres - SVM kernel = ‘rbf’

| Paramètre ɣ(gamma)  / Variable C  | Ɣ=1E-03 | Ɣ=1E-01 | Ɣ=1.0 | Ɣ=10.0 |
|------|------|------|------|------|
|   1E-03 | X | X | X | X |
|   1E-01 | X | X | X | X |
|   1.0 | X | X | X | X |
|   10.0 | X | X | X | X |

Note sur SVM:
3. Vous pouvez utiliser la classe sklearn.model_selection.GridSearchCV afin d’effectuer la recherche des meilleurs hyperparamètres. De ce fait, vous n’aurez pas besoin de changer manuellement les valeurs à chaque exécution. De plus, vous pouvez utiliser la classe sklearn.model_selection. StratifiedShuffleSplit afin de diviser l’ensemble de données en un n d’ensembles afin d’effectuer une validation croisée, le tout de manière automatique et aléatoire. Pour plus d’information, fiez-vous à l’exemple donné à la page RBF SVM Parameters disponible en annexe.

4. De plus, vous pouvez augmenter la vitesse d’exécution en parallélisant la recherche des hyperparamètres. En effet, la fonction GridSearchCV possède l’argument n_jobs=X où X est le nombre d’exécutions parallèle désiré (généralement proportionnel au nombre de coeurs x86 de la machine sur lequel s’exécute le code Python). Aussi, assurez-vous que votre appel de méthode spécifie une taille de cache suffisante pour le noyau SVM. Par l’intermédiaire du paramètre cache_size, vous serez en mesure de spécifier la taille de la cache désirée, ce qui peut réduire considérablement le temps de résolution de l’optimisation quadratique du problème. Ainsi, en spécifiant cache_size=2048 dans la déclaration de votre classificateur, vous spécifiez que l’algorithme peut utiliser jusqu’à 2 Go de mémoire vive [RAM] à titre de cache pour le noyau. Attention : votre ordinateur doit avoir suffisamment de mémoire vive. Assurez-vous de ne pas spécifier une valeur trop haute pour ne pas saturer la mémoire vive de votre ordinateur.

• Notez bien tous les résultats de la précision [accuracy] et le score F1 de chaque modèle calculé. Vous
devrez représenter ces résultats dans un tableau dans votre rapport. Prenez en note le temps d’exécution de vos modèles autant en phase d’apprentissage qu’en phase de test.

## Version courte 
1. Utiliser sklearn.model_selection.GridSearchCV pour trouver les meilleurs hyperparametres
2. sklearn.model_selection
3. StratifiedShuffleSplit afin de diviser l’ensemble de données en un n d’ensembles afin d’effectuer une validation croisée
4. Pour chaque modele calculé noté:
    1. Accuracy
    2. score F1
    3. Temps d'execution en phase apprentissage et en phase test

In [12]:
df = read_csv(r"C:\Users\gaspa\OneDrive\Bureau\COURS\ING 5\Semestre 1\Machine Learning\GTI770\labs\galaxy_feature_vectors.csv")

# get the train and test sample
df = df.drop(0, axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(df.loc[:, df.columns != 75], df[75], test_size=0.2,stratify=df[75])

In [19]:
def SVM(X_train,Y_train,X_test,Y_test,k,c,weight,g) :
    """
    Args:
        X_train (list): the train sample
        X_test (list): the test sample
        Y_train (list): the train output
        Y_test (list): the test output
    Returns:
        
    """
    acc_scorer = make_scorer(accuracy_score)
    f1_scorer = make_scorer(f1_score)
    scores = {'F1': f1_scorer, 'Accuracy': acc_scorer}

    # scale the data : réduire le execution time
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train = scaling.transform(X_train)
    X_test = scaling.transform(X_test)
    print("Debut du grid search")
    # implemente le gridsearchcv
    parameters = {'class_weight':weight,'cache_size':[6000],'C':[0.001,0.1,1,10],'gamma':[0.001,0.1,1,10],'kernel':('linear', 'rbf')}
    #parameters = {'class_weight':weight,'cache_size':[6000],'C':[0.001,0.1,1,10],'kernel':['linear']}
    model  = svm.SVC()
    clf = GridSearchCV(model, parameters, scoring=scores,refit='Accuracy',n_jobs=2,cv=3,return_train_score=True)
    clf.fit(X_train, Y_train)
    
    # sort le best model pour fit et score dessus
    best_model = clf.best_estimator_
    print(best_model)
    
    return clf

grid = SVM(X_train,Y_train,X_test,Y_test,'linear',10.0,['balanced'],'auto')

Debut du grid search
SVC(C=10, cache_size=6000, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [24]:
df = pd.DataFrame(grid.cv_results_)
df1 = df[['param_kernel','param_C','param_gamma','mean_test_F1','std_test_F1','mean_test_Accuracy','std_test_Accuracy','rank_test_Accuracy','mean_fit_time','std_fit_time','mean_score_time','std_score_time']]
df1 = df1.sort_values(['param_kernel','param_C','param_gamma'], ascending=[True, False, False])
df1

,param_kernel,param_C,param_gamma,mean_test_F1,std_test_F1,mean_test_Accuracy,std_test_Accuracy,rank_test_Accuracy,mean_fit_time,std_fit_time,mean_score_time,std_score_time
30,linear,10,10,0.943794,0.001819,0.941964,0.001815,5,13.844085,1.157593,1.052269,0.090952
28,linear,10,1,0.943794,0.001819,0.941964,0.001815,5,12.021735,1.461690,0.967000,0.114822
26,linear,10,0.1,0.943794,0.001819,0.941964,0.001815,5,11.294003,0.528975,0.904012,0.009436
24,linear,10,0.001,0.943794,0.001819,0.941964,0.001815,5,11.250313,1.156103,0.953658,0.102751
22,linear,1,10,0.941158,0.002491,0.939302,0.002534,9,5.032995,0.126540,1.053349,0.021622
20,linear,1,1,0.941158,0.002491,0.939302,0.002534,9,4.705361,0.114248,1.077988,0.034324
18,linear,1,0.1,0.941158,0.002491,0.939302,0.002534,9,4.711354,0.076673,1.085313,0.016436
16,linear,1,0.001,0.941158,0.002491,0.939302,0.002534,9,4.859334,0.218558,1.083661,0.042348
14,linear,0.1,10,0.935251,0.002460,0.933240,0.002479,14,5.352372,0.132732,1.629639,0.036515
12,linear,0.1,1,0.935251,0.002460,0.933240,0.002479,14,5.408309,0.158492,1.602026,0.031360


In [28]:
# scale the data : réduire le execution time
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
xtrain = scaling.transform(X_train)
xtest = scaling.transform(X_test)

# afficher le meilleur modèle et le stocker
best_model = grid.best_estimator_
best_model.fit(xtrain, Y_train)
print(best_model)
print("\n\n")
print("Score of the best hyperparametered model:   " + str(best_model.score(xtest,Y_test)))

SVC(C=10, cache_size=6000, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)



Score of the best hyperparametered model:   0.9636309875813128


# Rapport

### Introduction

### Question 1 - 
Parmi les méthodes de validation (Leave-one-out cross-validation, Leave-p-out crossvalidation, k-fold cross-validation), présentez l’approche de validation que vous avez utilisée et pourquoi vous l’avez utilisée. Faites des liens avec les modèles d’apprentissage à l’étude. Rappelez-vous que vous pouvez sélectionner plus qu’une méthode qui servira à la validation des deux modèles à l’études dans ce laboratoire

### Question 2 - 
Décrivez la méthode de normalisation de donnés utilisée. 

### Question 3 - 
Décrivez la structure et le choix de votre modèle d’apprentissage MLP. Faites un parallèle entre votre description et votre implémentation. Quelle est la fonction de coût utilisée ? Pourquoi avez-vous utilisé cette fonction plutôt qu’une autre (quels sont les avantages et inconvénients ?) 

### Question 4 - 
Avec les graphiques créés par TensorBoard (TensorFlow) ou Visdom (PyTorch), après combien d’itérations/epochs êtes-vous en état de surapprentissage [overfitting]? À votre avis, quel est le nombre d’epochs optimal pour votre modèle?

### Question 5 - 
Pour chacun des modèles d’apprentissage élaborés, présentez sous forme de tableau et de graphique les résultats de la précision (accuracy) et du score F1. Expliquez l’impact des hyperparamètres sur les performances du modèle MLP. Comment se traduisent les divers changements de paramètres ou pourquoi ont-il cet impact sur la performance du modèle ? Prenez en note le temps d’exécution de vos modèles autant en phase d’apprentissage qu’en phase de test.

### Question 6 - 
Présentez brièvement la méthode que vous avez utilisée afin de trouver le meilleur modèle SVM. Quels ont été vos résultats? Quels sont les impacts des hyperparamètres et leur utilité respective?

### Question 7 - 
Quel est l’impact de la taille de l’ensemble d’apprentissages sur la performance de classification des différents modèles?

### Question 8 - 
Quel type de classificateur recommanderiez-vous pour l’ensemble de données des
galaxies et dans quelles conditions (par exemple mais non exhaustif, le nombre de données privilégié, les hyperparamètres, le temps de calcul, le matériel nécessaire, les scores de performance)? Discutez des performances que vous avez obtenues entre les modèles d’apprentissage utilisant les réseaux de neurones et les modèles SVM.

### Question 9 - 
Formulez quelques pistes d’amélioration des classificateurs. 

### Conclusion